[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ManuelEspejo/Machine-Learning-Bases/blob/main/notebooks/01_Aprendizaje-Supervisado.ipynb) 👈🏻​ **Pulsar para abrir en Colab​**

# ¿Cómo usar estos notebooks?

Si este es el primer notebook que abres en este repositorio, te recomiendo que antes leas el [Manual de uso de estos notebooks](https://github.com/ManuelEspejo/Machine-Learning-Bases/blob/main/docs/manual-notebooks.md) que he creado para que te familiarices con el proyecto y las distintas rutas que puedes seguir, luego puedes volver aquí y continuar.

En este notebook, vamos a profundizar en par de ejemplos prácticos de aprendizaje supervisado.

Por otra parte, si aún no has revisado el notebook "[00_Empieza-aquí.ipynb](https://github.com/ManuelEspejo/Machine-Learning-Bases/blob/main/notebooks/00_Empieza-aquí.ipynb)", te sugiero que le eches un vistazo primero para conocer los conceptos básicos. Pero si ya tienes una idea clara de qué es el aprendizaje no supervisado y quieres verlo en acción, ¡estás en el lugar correcto!

# 01. Aprendizaje No Supervisado

El **Aprendizaje No Supervisado** es un tipo de Machine Learning donde el modelo trabaja con datos que no tienen etiquetas predefinidas para encontrar patrones, relaciones y estructuras en los datos. Es como explorar un bosque sin un mapa, dejando que los datos "hablen por sí mismos".

- En el aprendizaje supervisado, sabes cuál es la respuesta correcta (datos etiquetados).
- En el aprendizaje no supervisado, simplemente intentas encontrar una estructura significativa en los datos.

## ¿Por qué es importante?

En muchos casos del mundo real, los datos no vienen con etiquetas. Imagina los millones de interacciones de clientes en un sitio web: sería inviable etiquetar manualmente cada una. Aquí es donde entra el aprendizaje no supervisado para ayudarnos a:

- Agrupar clientes con comportamientos similares (clustering).
- Reducir la complejidad de datos con cientos de variables (reducción de dimensionalidad).
- Identificar comportamientos anómalos, como transacciones fraudulentas (detección de anomalías).

## ¿Qué vamos a ver?

A lo largo de este notebook, exploraremos las técnicas más comunes de aprendizaje no supervisado:

- **Clustering con K-Means**: Cómo agrupar datos según su similitud.
- **Reducción de Dimensionalidad con PCA**: Cómo simplificar datos complejos sin perder información clave.
- **Detección de Anomalías**: Cómo identificar datos que no siguen patrones normales.

> **📚 Recurso:** Si aún no has explorado el aprendizaje supervisado, te recomiendo revisar el notebook anterior: [01_Aprendizaje-Supervisado.ipynb](https://github.com/ManuelEspejo/Machine-Learning-Bases/blob/main/notebooks/01_Aprendizaje-Supervisado.ipynb). Esto te ayudará a entender mejor las diferencias y complementar ambas técnicas en tus proyectos.


**Empecemos**🚀

# Índice de Contenidos

1. **Clustering: Segmentación de Clientes**
   
   1.1. Descripción del Problema  
   1.2. Descargar el Dataset  
   1.3. Exploración y Preprocesamiento de Datos  
   1.4. Selección de Características  
   1.5. Escalado de los Datos  
   1.6. Visualización Inicial
   1.7. Aplicar K-Means  
   1.8. Evaluar la Calidad de los Clusters  
   1.9. Visualización de Clusters  
   1.10. Reflexiones y Mejoras  

2. **Reducción de Dimensionalidad: Visualización de Datos con PCA**
   
   2.1. Introducción a PCA  
   2.2. Aplicar PCA a los Datos  
   2.3. Visualizar los Resultados en 2D  
   2.4. Interpretación de los Componentes Principales  
   2.5. Reflexiones sobre el uso de PCA  

3. **Detección de Anomalías: Identificación de Transacciones Atípicas**
   
   3.1. Descripción del Problema  
   3.2. Exploración de Datos  
   3.3. Aplicar un Modelo de Detección de Anomalías  
   3.4. Visualización de los Resultados  
   3.5. Reflexiones sobre la Interpretación de Anomalías  

4. **Exploraciones Avanzadas**
   
   4.1. Combinación de Clustering y PCA  
   4.2. Otros Algoritmos de Clustering (DBSCAN)  
   4.3. Visualización con T-SNE y UMAP  

5. **Ejercicios y Retos**
   
   5.1. Clustering con Datos Reales  
   5.2. Implementación de PCA en un Dataset Propio  
   5.3. Experimentar con Detección de Anomalías  
   5.4. Preguntas para Reflexionar  

